In [1]:
from collections import Counter
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import os

In [2]:
# Define root directory
# root = os.path.dirname(os.path.abspath(__file__))
root = "."
sys.path.append(os.path.join(root, "..", "src"))
from default import DATAPATH, CONFIGPATH

# Loading ChEMBL preprocessed data
print("Loading ChEMBL preprocessed data...")
ChEMBL = pd.read_csv(os.path.join(DATAPATH, "chembl_processed", "activities_preprocessed.csv"), low_memory=False)

# Filtering data for pathogens
print("Filtering data for pathogens...")
# List of pathogens
pathogens = ["Acinetobacter baumannii", "Candida albicans", "Campylobacter", "Escherichia coli", "Enterococcus faecium", "Enterobacter",
             "Helicobacter pylori", "Klebsiella pneumoniae", "Mycobacterium tuberculosis", "Neisseria gonorrhoeae", "Pseudomonas aeruginosa",
             "Plasmodium falciparum", "Staphylococcus aureus", "Schistosoma mansoni", "Streptococcus pneumoniae"]
pathogen_data = ChEMBL[ChEMBL['target_organism'].str.lower().isin([p.lower() for p in pathogens]) | 
                       ChEMBL['assay_organism'].str.lower().isin([p.lower() for p in pathogens])].reset_index(drop=True)

print(f"Number of activities: {len(pathogen_data)}")
print(f"Number of activities with non nan values: {len(pathogen_data[pathogen_data['value'].isna() == False])}")
print(f"Number of unique compounds: {len(set(pathogen_data['compound_chembl_id']))}")
print(f"Number of unique assays: {len(set(pathogen_data['assay_chembl_id']))}")
print(f'Assay types: {Counter(pathogen_data[["assay_chembl_id", "assay_type"]].drop_duplicates(subset="assay_chembl_id")["assay_type"])}')
print(f"Number of unique targets: {len(set(pathogen_data['target_chembl_id']))}")
print(f'Target types: {Counter(pathogen_data[["target_chembl_id", "target_type"]].drop_duplicates(subset="target_chembl_id")["target_type"])}')
print(f"Relations: {Counter(pathogen_data['relation'])}")
print(f"Original pChEMBL: {len(pathogen_data[pathogen_data['pchembl'].isna() == False])}")
print(f"Calculated pChEMBL: {len(pathogen_data[pathogen_data['pchembl_calculated'].isna() == False])}")

thr = 0.01
pathogen_data_filtered = pathogen_data[(pathogen_data['pchembl'].isna() == False) & (pathogen_data['pchembl_calculated'].isna() == False)].copy()
pathogen_data_filtered['pchembl'] = pathogen_data_filtered['pchembl'].clip(lower=1, upper=9)
pathogen_data_filtered = pathogen_data_filtered[["pchembl", "pchembl_calculated"]]
pathogen_data_filtered['difference'] = [np.abs(i-j) for i,j in zip(pathogen_data_filtered['pchembl'], pathogen_data_filtered['pchembl_calculated'])]
perc = len(pathogen_data_filtered[pathogen_data_filtered['difference'] < thr]) * 100 / len(pathogen_data_filtered)
print(f"Percentage of pChEMBL original vs calculated having difference < {thr}: {perc}")

# Get directions
DIRECTIONS = pd.read_csv(os.path.join(CONFIGPATH, 'activity_std_units_curated_manual_curation.csv'))
DIRECTIONS = {(i,j): k for i,j,k in zip(DIRECTIONS['activity_type'], DIRECTIONS['unit'], DIRECTIONS['manual_curation_direction']) if np.isnan(k) == False}

s = pathogen_data[["activity_type", "unit", 'target_type']]
out = (
s.value_counts(subset=["activity_type", "unit", "target_type"], dropna=False)
    .reset_index(name="count")
    .sort_values("count", ascending=False, ignore_index=True))

out['direction'] = [DIRECTIONS[(i,j)] if (i,j) in DIRECTIONS else np.nan for i,j in out[["activity_type", "unit"]].values]
print(f"Number of activities (15 pathogens) having an [act_type, unit] pair without an assigned direction: {sum(out[out['direction'].isna() == True]['count'])}")
missing_direction = out[out['direction'].isna() == True].reset_index(drop=True)
total_count = missing_direction['count'].sum()
missing_direction['cumulative_prop'] = (missing_direction['count'].cumsum() / total_count).round(3)
out = out[out['direction'].isna() == False].reset_index(drop=True)
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)

Loading ChEMBL preprocessed data...
Filtering data for pathogens...
Number of activities: 2725613
Number of activities with non nan values: 2610850
Number of unique compounds: 710802
Number of unique assays: 128689
Assay types: Counter({'F': 111241, 'B': 16005, 'A': 1273, 'T': 151, 'U': 14, 'P': 5})
Number of unique targets: 798
Target types: Counter({'SINGLE PROTEIN': 724, 'ORGANISM': 23, 'PROTEIN COMPLEX': 17, 'PROTEIN FAMILY': 8, 'NUCLEIC-ACID': 5, 'MACROMOLECULE': 4, 'SUBCELLULAR': 4, 'SMALL MOLECULE': 2, 'PROTEIN COMPLEX GROUP': 2, 'CELL-LINE': 2, 'UNCHECKED': 1, 'NON-MOLECULAR': 1, 'ADMET': 1, 'NO TARGET': 1, 'PROTEIN NUCLEIC-ACID COMPLEX': 1, 'UNKNOWN': 1, 'LIPID': 1})
Relations: Counter({'=': 2550060, '>': 155126, '<': 20427})
Original pChEMBL: 220565
Calculated pChEMBL: 1117735
Percentage of pChEMBL original vs calculated having difference < 0.01: 100.0
Number of activities (15 pathogens) having an [act_type, unit] pair without an assigned direction: 356198


In [4]:
PREDEFINED_CUTOFFS = {
    ("MIC", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("MIC", "umol.L-1", "SINGLE PROTEIN"): [0.5, 1, 5],
    ("MIC50", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("MIC80", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("MIC90", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("MIC99", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("IC50", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("IC50", "umol.L-1", "SINGLE PROTEIN"): [0.5, 1, 5],
    ("AC50", "umol.L-1", "SINGLE PROTEIN"): [0.5, 1, 5],
    ("IC90", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("EC50", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("EC50", "umol.L-1", "SINGLE PROTEIN"): [0.5, 1, 5],
    ("ED50", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("POTENCY", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("POTENCY", "umol.L-1", "SINGLE PROTEIN"): [0.5, 1, 5],
    ("KI", "umol.L-1", "SINGLE PROTEIN"): [0.5, 1, 5],
    ("ACTIVITY", "umol.L-1", "ORGANISM"): [5, 10, 20],
    ("INHIBITION", "%", "ORGANISM"): [25, 50, 75],
    ("INHIBITION", "%", "SINGLE PROTEIN"): [25, 50, 75],
    ("PERCENTEFFECT", "%", "SINGLE PROTEIN"): [25, 50, 75],
    ("PERCENTEFFECT", "%", "ORGANISM"): [25, 50, 75],
    ("ACTIVITY", "%", "SINGLE PROTEIN"): [25, 50, 75],
    ("ACTIVITY", "%", "ORGANISM"): [25, 50, 75],
    ("GI", "%", "ORGANISM"): [25, 50, 75],
    ("IZ", "mm", "ORGANISM"): [5, 10, 20],
}

def get_pathogen_code(pathogen):
    return str(pathogen.split()[0][0] + pathogen.split()[1]).lower() if len(pathogen.split()) > 1 else pathogen.lower()

EXPERT_CUTOFFs = []
for activity_type, unit, target_type, direction in out[['activity_type', 'unit', 'target_type', 'direction']].values[:30]:
    if unit not in ['ug.mg-1']:
        for pathogen in pathogens:
            if target_type in ['SINGLE PROTEIN', 'ORGANISM']:
                if (activity_type, unit, target_type) in PREDEFINED_CUTOFFS:
                    cutoff = ";".join([str(k) for k in PREDEFINED_CUTOFFS[(activity_type, unit, target_type)]])
                else:
                    cutoff = np.nan
                EXPERT_CUTOFFs.append([activity_type, unit, target_type, direction, get_pathogen_code(pathogen), cutoff])

EXPERT_CUTOFFs = pd.DataFrame(EXPERT_CUTOFFs, columns=['activity_type', 'unit', 'target_type', 'direction', 'pathogen_code', 'expert_cutoffs'])
EXPERT_CUTOFFs.to_csv(os.path.join(CONFIGPATH, "expert_cutoffs_.csv"), index=False)

In [14]:
sum(out[out['direction'] != 0]['count'])

2368357

In [28]:
out

,activity_type,unit,target_type,count,direction,cumulative_prop
0,MIC,umol.L-1,ORGANISM,450846,-1.0,0.190
1,INHIBITION,%,ORGANISM,436254,1.0,0.374
2,POTENCY,umol.L-1,ORGANISM,302212,-1.0,0.502
3,PERCENTEFFECT,%,SINGLE PROTEIN,285312,1.0,0.622
4,PERCENTEFFECT,%,ORGANISM,197170,1.0,0.706
...,...,...,...,...,...,...
144,KI,umol.L-1,PROTEIN COMPLEX GROUP,1,-1.0,1.000
145,KI,umol.L-1,NUCLEIC-ACID,1,-1.0,1.000
146,T12,s,ADMET,1,0.0,1.000
147,ACTIVITY,%,PROTEIN NUCLEIC-ACID COMPLEX,1,1.0,1.000


In [33]:
sum([i[3] for i in out.values if tuple(i[:3]) in PREDEFINED_CUTOFFS])

2041980

In [34]:
2041980 / 2368357 * 100

86.21926508545798